# Multivariate Linear Regression

### Theoretical Overview<br>
$$ H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3 + b $$
$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$<br>
$H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가<br>
$cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

### Imports

In [8]:
import torch

### Data

In [10]:
#실습을 위해 Naive한 Matrix 형태의 Data 이용
#dataset 생성 3번의 쪽지시험 점수(x) 를 통한 최종시험(y) 점수 예측
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

### Training

In [11]:
# Model 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# Optimize 설정
optimizer = torch.optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    #H(x) = w1x1 + w2x2 + ... + wnxn으로 갯수가 늘어남에 따라 term이 너무 길어진다. 따라서 torch 함수를 사용
    hypothesis = x_train.matmul(W) + b # or .mm or @
    
    #Cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #squeeze()(unsqueeze()) 차원의 원소가 1인 차원을 제거. detach() 미분값을 계산하기 싫은 경우 사용 
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))


Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.713135
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371017
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

12회 이후로는 Cost값의 변화가 크게 줄어든다. 

* MultivariateLinearRegressionModel class에서 nn 모듈을 상속하고 nn을 쉽게 만들 수 있게 해준다.

import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)
        
hypothesis = model(x_train)

* torch에서 제공하는 cost function
import torch.nn.functional as F
mse_loss(prediection, y_train)


torch에서 제공하는 library를 통해 code를 수정

### High-level Implementation with nn.Module

### Imports

In [5]:
import torch.nn as nn
import torch.nn.functional as F

### Class 생성

In [6]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

### Training

In [9]:
#dataset 생성 3번의 쪽지시험 점수(x) 를 통한 최종시험(y) 점수 예측
x_train = torch.FloatTensor([[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# Model 초기화
#W = torch.zeros((3, 1), requires_grad=True)
#b = torch.zeros(1, requires_grad=True)
model = MultivariateLinearRegressionModel()

# Optimize 설정
#optimizer = torch.optim.SGD([W, b], lr=1e-5)
#W, b 대신 model에 MLRM class를 사용했으므로 해당 parameter를 불러오는 함수를 입력한다.
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    #H(x) = w1x1 + w2x2 + ... + wnxn으로 갯수가 늘어남에 따라 term이 너무 길어진다. 따라서 torch 함수를 사용
    #hypothesis = x_train.matmul(W) + b # or .mm or @
    prediction = model(x_train)
    
    #Cost 계산
    #cost = torch.mean((hypothesis - y_train) ** 2)
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, prediction.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([70.2414, 88.5342, 85.1069, 93.4358, 67.8106]) Cost: 8203.661133
Epoch    1/20 hypothesis: tensor([105.6122, 131.0472, 126.9957, 139.0516, 100.2372]) Cost: 2571.887939
Epoch    2/20 hypothesis: tensor([125.4151, 154.8486, 150.4477, 164.5902, 118.3917]) Cost: 806.625305
Epoch    3/20 hypothesis: tensor([136.5020, 168.1741, 163.5777, 178.8884, 128.5557]) Cost: 253.309662
Epoch    4/20 hypothesis: tensor([142.7093, 175.6345, 170.9286, 186.8933, 134.2461]) Cost: 79.874825
Epoch    5/20 hypothesis: tensor([146.1845, 179.8113, 175.0442, 191.3751, 137.4320]) Cost: 25.511887
Epoch    6/20 hypothesis: tensor([148.1302, 182.1497, 177.3484, 193.8842, 139.2156]) Cost: 8.472000
Epoch    7/20 hypothesis: tensor([149.2195, 183.4589, 178.6384, 195.2889, 140.2141]) Cost: 3.130989
Epoch    8/20 hypothesis: tensor([149.8295, 184.1918, 179.3607, 196.0754, 140.7732]) Cost: 1.456769
Epoch    9/20 hypothesis: tensor([150.1710, 184.6021, 179.7650, 196.5157, 141.0861]) Cost: 0.